In [1]:
!git clone https://github.com/saprmarks/dictionary_learning
!cd dictionary_learning && pip install -r requirements.txt
%pip install zstandard datasets nnsight

Cloning into 'dictionary_learning'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 157 (delta 12), reused 17 (delta 8), pack-reused 134
Receiving objects: 100% (157/157), 263.47 KiB | 4.70 MiB/s, done.
Resolving deltas: 100% (86/86), done.
  Using cached nnsight-0.1.15-py3-none-any.whl (3.5 MB)
  Using cached pydantic-2.5.3-py3-none-any.whl (381 kB)
  Using cached python_socketio-5.11.0-py3-none-any.whl (75 kB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached diffusers-0.25.1-py3-none-any.whl (1.8 MB)
  Using cached torchvision-0.16.2-cp310-cp310-manylinux1_x86_64.whl (6.8 MB)
  Using cached annotated_types-0.6.0-py3-none-any.whl (12 kB)
  Using cached pydantic_core-2.14.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Using cached importlib_metadata-7.0.1-py3-none-any.whl (23 kB)
  Using cac

In [2]:
from datasets import load_dataset

dataset = load_dataset("jbrinkma/tinystories-v2-clean")
dataset

/home/jacob/dev/transformer-playground/.direnv/python-3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 2705118/2705118 [00:05<00:00, 500906.48 examples/s]


DatasetDict({
    validation: Dataset({
        features: ['story'],
        num_rows: 27516
    })
    train: Dataset({
        features: ['story'],
        num_rows: 2705118
    })
})

In [8]:
train_data = dataset['train']['story']
train_data_iter = iter(train_data)

'Sam and Lily are friends. They like to play in the park. One day, they see a big dog. The dog is brown and shiny. He has a collar with a tag. The tag says "Copper".\n"Look, Lily, a dog!" Sam says. "He is so pretty. Can we pet him?"\n"I don\'t know, Sam," Lily says. "Maybe he is not friendly. Maybe he will bite us."\nSam and Lily are scared. They want to run away. But Copper is not a bad dog. He is a good dog. He is a dependable dog. He likes children. He wags his tail and barks softly.\n"Hello, Copper," a voice says. "Are you making new friends?"\nSam and Lily turn around. They see a man. The man is Copper\'s owner. He smiles at them.\n"Don\'t be afraid," the man says. "Copper is very gentle. He loves to be petted. Do you want to say hello to him?"\nSam and Lily nod. They walk slowly to Copper. They touch his fur. It is soft and warm. Copper licks their hands. He is happy.\n"Wow, he is so nice," Sam says. "Can we play with him?"\n"Sure, you can," the man says. "But only if your parent

In [9]:
from nnsight import LanguageModel
from dictionary_learning.buffer import ActivationBuffer
from dictionary_learning.training import trainSAE

model = LanguageModel(
    'jbrinkma/delphi-llama2-100k', # this can be any Huggingface model
)
submodule = model.model.layers[1].mlp # layer 1 MLP
activation_dim = 48 # output dimension of the MLP
dictionary_size = 4 * activation_dim

buffer = ActivationBuffer(
    train_data_iter,
    model,
    submodule,
    out_feats=activation_dim, # output dimension of the model component
) # buffer will return batches of tensors of dimension = submodule's output dimension

# train the sparse autoencoder (SAE)
ae = trainSAE(
    buffer,
    activation_dim,
    dictionary_size,
    lr=3e-4,
    sparsity_penalty=1e-3,
)

0it [00:00, ?it/s]You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
pytorch_model.bin: 100%|██████████| 1.21M/1.21M [00:00<00:00, 26.7MB/s]
2it [01:23, 34.24s/it]

step 0 MSE loss: 0.12227676063776016, sparsity loss: 6.593033790588379


1002it [06:19,  7.80it/s]

step 1000 MSE loss: 0.0026439388748258352, sparsity loss: 4.406581878662109


1247it [07:30,  2.77it/s]


KeyboardInterrupt: 